## 导入所需要的包
#### 就是导入机器学习所需要得包，可以使用pip或者conda命令来安装这些包，但是建议使用同一个，要么一直用pip，要么一直用conda。并且在安装之前可以先询问一下AI所需要安装的版本，不然可能到时候因为版本不兼容而报错

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import axes
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

## 1. 数据读取
#### 读取一下数据，数据集分为训练集和测试集。训练集呢是拿来让模型学习的，让模型拿来确定各种参数。测试集呢就是拿来验证这个模型的效果。
#### 我觉得要进行机械学习一开始最重要的就是要了解你的数据集，要知道你的数据集包含哪些特征，还要知道特征的形式，这些都是很重要的，非常忌讳不看数据就直接在那里开始着手训练，这样也是云里雾里的。本次获得的一些是关于银行的数据，这是一个关于预测用户去留的问题，0表示的就是存留，1表示的就是客户流失。其实我觉得了解一些这些背景也是很重要的，这样你就大概可以初步的知道哪些特征重要，哪些特征不重要，可以对特征进行初步的筛选。

In [2]:
train_df=pd.read_csv("./data/train.csv")
test_df=pd.read_csv("./data/test.csv")
# 2. 看看数据的样子
print("训练集形状 (Rows, Cols):", train_df.shape)
print("\n训练集前 5 行：")
print(train_df.head()) 
train_df.info()

训练集形状 (Rows, Cols): (15000, 14)

训练集前 5 行：
   id  CustomerId       Surname  CreditScore Geography  Gender   Age  Tenure  \
0   0  15709511.0        Ch'ang        754.0     Spain    Male  40.0     8.0   
1   1  15592761.0      Genovese        579.0    France  Female  28.0     1.0   
2   2  15579914.0           Yeh        744.0    France  Female  56.0     5.0   
3   3  15669611.0  Nwachinemelu        697.0    France    Male  29.0     2.0   
4   4  15761775.0         Hs?eh        628.0    France  Female  22.0     9.0   

     Balance  NumOfProducts  HasCrCard  IsActiveMember  EstimatedSalary  \
0  102954.68            2.0        1.0             1.0        149238.35   
1       0.00            2.0        1.0             0.0         64869.32   
2       0.00            1.0        1.0             0.0        158816.03   
3       0.00            2.0        1.0             0.0         55775.72   
4       0.00            2.0        1.0             0.0         49653.39   

   Exited  
0     0.0  
1

## 2. 数据清理

#### 数据清理是一个很重要的步骤，数据的好坏直接的决定了你的模型的上限。
#### 主要就包括缺失值的处理，异常值的处理。如果是缺失值我们可以尝试使用众数或者平均数来填充，当然这取决于具体的特征。如果是异常值的话我们就需要修改这些异常值，不然这些异常就会作为噪声对我们的模型学习产生巨大的影响。
#### 本次训练只检查了缺失值，并没有检查异常值，并且缺失值的个数为0。因为这个数据集是为新手准备的练手数据集，所以很简单。但是实际上需要处理的数据集都是很麻烦的。

In [3]:
# --- 1. 检查缺失值 ---
print("训练集缺失值统计：")
print(train_df.isnull().sum())
# --- 2. 缺失值填充策略 ---
# 对于数值型变量（年龄、薪水等），如果缺失，用“中位数”填充是最稳妥的（防止被极值拉偏）
# 对于分类变量（地理位置等），如果有缺失，可以用“众数”填充

训练集缺失值统计：
id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64


## 3. 特征工程

#### 特征工程就是对这些特征进行一系列的处理，包括添加新特征，特征编码等步骤，接下来我做的就是进行新特征的添加。
#### 为什么要进行特征工程呢？目的就是为了使我们的模型预测效果更好，如果我们在老特征上面进行衍生添加了新特征，这个新特征对我们的模型预测室友效果的，那说明这个特征工程还是做得有意义的。
#### 这一部分是通过已有特征来进行计算得到的新特征。一个是账户余额/薪资，一个是忠诚度/年龄比，这是个人感觉可能这两个新特征会对客户去留产生影响。

In [4]:
#账户余额/薪资
#比值越高说明更倾向于存钱
train_df['BalanceSalaryRatio'] = train_df['Balance'] / (train_df['EstimatedSalary'] + 1e-5)
test_df['BalanceSalaryRatio'] = test_df['Balance'] / (test_df['EstimatedSalary'] + 1e-5)

# 2. 忠诚度/年龄比 (Tenure / Age)
# 逻辑：同样的 5 年老客户，20岁的人（占了他1/4的人生）比 50岁的人（只占1/10）对银行的情感依赖可能更重。
train_df['TenureByAge'] = train_df['Tenure'] / (train_df['Age'] + 1e-5)
test_df['TenureByAge'] = test_df['Tenure'] / (test_df['Age'] + 1e-5)

#### 接下来是进行了一个特征重要性的判断，就是看哪些特征跟y值（用户去留）相关。负值呢代表是对客户留存起到了积极的作用，正值则是导致客户流失的原因，可以看到年龄是对这个y值影响最大的，我们创造的一个特征忠诚度/年龄比(Tenure / Age)对这个也有不小的影响（因为公式中有年龄，受了年龄不小的影响）。但是另外一个创造的新特征账户余额/薪资（BalanceSalaryRatio）确影响很小。一个好的特征工程可以直接影响最后模型的效果。

In [5]:
numeric_cols = ['Age', 'CreditScore', 'Balance', 'EstimatedSalary', 'Tenure', 
                'BalanceSalaryRatio', 'TenureByAge']
correlations = train_df[numeric_cols + ['Exited']].corr()['Exited'].drop('Exited').sort_values()
print(correlations)

TenureByAge          -0.162805
CreditScore          -0.045404
Tenure               -0.015022
BalanceSalaryRatio   -0.002198
EstimatedSalary       0.023511
Balance               0.155268
Age                   0.481368
Name: Exited, dtype: float64


## 4. 特征编码

#### 这里主要是关于特征编码的部分。为什么要进行特征编码呢？是因为机器学习究其本质也是数学的运算，所以有些字符特征不能直接给模型，要先把他量化过后再给模型。我们这里组要介绍两种方法，独热编码和标签编码。
#### 标签编码接把文字映射成整数，但是这种方法会让模型认为数值大的一个类别较大，影响模型判断。
#### 在许多特征当中，就比如这次需要编码的特征，一个是地区，一个是性别，类别之间没有大小关系，且类别数量不多。这个就适合独热编码。这样他就不会因为数值的大小而认为编码的特征也有大小。
#### 当然这里我们还对特征进行筛选，就像我们前面提到的那样，对背景有一定了解，比如我们知道用户ID和名字这一类的特征对用户去留并没有影响，所以我们就可以直接把他们筛掉，这些对我们模型训练没有一点用处，当然，对于哪些对y值不重要的特征我也一样筛除掉了。
#### 在这里我也对训练集和验证集进行了划分按照8：2的方法进行划分（其实可以使用k-fold这种方法更加的先进）。划分训练集和验证集的核心目的是为了防止过拟合并评估模型的泛化能力；训练集用于让模型拟合参数、学习数据的内在规律（即“学习”），而验证集则作为模型从未见过的独立样本（即“模拟考”），用于实时监控模型对未知数据的预测水平，从而指导超参数调整并触发早停机制（Early Stopping），确保模型不仅能“记住”旧数据，更能“读懂”新数据。

In [6]:
encoder = LabelEncoder()
categorical_cols = ["Gender", "Geography"]

for feature in categorical_cols:
    train_df[feature] = encoder.fit_transform(train_df[feature])
    test_df[feature] = encoder.transform(test_df[feature])
display(train_df.head())




drop_cols = ['id', 'CustomerId', 'Surname', 'Exited','Tenure','BalanceSalaryRatio']
X = train_df.drop(columns=drop_cols)
y = train_df['Exited']
X_test_final = test_df.drop(columns=['id', 'CustomerId', 'Surname','Tenure','BalanceSalaryRatio'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test_final)
print(X_train.head())

print("数据准备完毕！")
print(f"训练集形状: {X_train_scaled.shape}")
print(f"验证集形状: {X_val_scaled.shape}")

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,BalanceSalaryRatio,TenureByAge
0,0,15709511.0,Ch'ang,754.0,2,1,40.0,8.0,102954.68,2.0,1.0,1.0,149238.35,0.0,0.689867,0.200000
1,1,15592761.0,Genovese,579.0,0,0,28.0,1.0,0.00,2.0,1.0,0.0,64869.32,0.0,0.000000,0.035714
2,2,15579914.0,Yeh,744.0,0,0,56.0,5.0,0.00,1.0,1.0,0.0,158816.03,1.0,0.000000,0.089286
3,3,15669611.0,Nwachinemelu,697.0,0,1,29.0,2.0,0.00,2.0,1.0,0.0,55775.72,0.0,0.000000,0.068965
4,4,15761775.0,Hs?eh,628.0,0,0,22.0,9.0,0.00,2.0,1.0,0.0,49653.39,0.0,0.000000,0.409091


      CreditScore  Geography  Gender   Age    Balance  NumOfProducts  \
9451        567.0          0       1  43.0       0.00            1.0   
2071        612.0          1       1  35.0  104498.79            1.0   
77          619.0          0       1  34.0       0.00            2.0   
5464        710.0          0       0  43.0       0.00            2.0   
8844        662.0          1       1  40.0  120165.40            1.0   

      HasCrCard  IsActiveMember  EstimatedSalary  TenureByAge  
9451        1.0             0.0         71843.95     0.186046  
2071        1.0             0.0         51112.80     0.114286  
77          1.0             1.0        141152.28     0.058824  
5464        1.0             1.0        175072.47     0.046512  
8844        1.0             1.0         72993.65     0.050000  
数据准备完毕！
训练集形状: (12000, 10)
验证集形状: (3000, 10)


## 4. 模型训练

#### 数据已经准好了，接下来就是对模型进行训练了，让模型来学习数据之中的规律了。
#### 这里我们使用了三个模型来进行训练，模型里面有许多参数要进行合理的设置，比如说什么学习率啊，什么树的最大深度这些都是要进行调整的，可以根据每次训练的结果来进行参数的调节。
#### 这里就是使用训练集训练好一套参数，然后在使用验证集来验证，然后根据结果再对参数不断的进行调以达到在验证集上面也有好效果。

In [10]:
# --- 1. 定义模型列表 ---
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=500, max_depth=10, random_state=42, n_jobs=-1),
    "XGBoost": XGBClassifier(n_estimators=1000, learning_rate=0.05, max_depth=5, eval_metric='auc', random_state=42, n_jobs=-1)
}

# --- 2. 循环训练并评估 ---
print(f"{'Model Name':<20} | {'Accuracy':<10} | {'AUC Score':<10}")
print("-" * 45)

results = {}

for name, model in models.items():
    # 训练
    model.fit(X_train_scaled, y_train)
    
    # 预测 (验证集)
    val_preds = model.predict(X_val_scaled)
    val_proba = model.predict_proba(X_val_scaled)[:, 1]
    
    # 计算指标
    acc = accuracy_score(y_val, val_preds)
    auc = roc_auc_score(y_val, val_proba)
    
    # 存起来方便后面融合
    results[name] = val_proba
    
    print(f"{name:<20} | {acc:.4f}     | {auc:.4f}")

Model Name           | Accuracy   | AUC Score 
---------------------------------------------
Logistic Regression  | 0.8623     | 0.8743
Random Forest        | 0.8933     | 0.9309
XGBoost              | 0.8903     | 0.9264


#### 可以打印出来一些中间变量出来看看，这些数据长什么样子的。

In [16]:
results['XGBoost'][:5]

array([0.6721171 , 0.00171207, 0.1837626 , 0.01838813, 0.06998032],
      dtype=float32)

#### 这里是简单的对三个模型进行了简单的加权融合（当然这也有更加科学的融合方法）。
#### 这个融合简单来说就是，每个模型对这个客户的去留都有自己的看法，然后让他们共同投票来判断这个人到底是否去留。当然因为权重的不同，所以每个模型说话的分量也是不同的。这里我们给了随机森林最重的话语权。

In [19]:
# --- 简单加权融合 ---
# XGBoost 最强，给大权重 (0.6)
# 随机森林 次之 (0.3)
# 逻辑回归 辅助 (0.1)

ensemble_pred = (0.1 * results['Logistic Regression'] + 
                 0.6 * results['Random Forest'] + 
                 0.3 * results['XGBoost'])

ensemble_auc = roc_auc_score(y_val, ensemble_pred)
print("\n---------------------------------------")
print(f">>> 模型融合 (Ensemble) AUC: {ensemble_auc:.4f}")
print("---------------------------------------")


---------------------------------------
>>> 模型融合 (Ensemble) AUC: 0.9320
---------------------------------------


#### 最后我们的模型就训练好了。
#### 现在就是在测试集上面大显身手了。之前在训练过程中测试集是完全没有出现的（在特征工程中出现过，因为需要把新特征也添加进去），所以这个模型只是学到了训练集里面的东西，现在测试集的东西对他来说都是新的，所以就可以考察这个模型到底怎样。

In [20]:
# --- 1. 让三个模型分别预测测试集 (Test Set) ---
# 注意：这里用的是 X_test_scaled (测试集数据)，不是 X_val_scaled
test_probs_lr  = models['Logistic Regression'].predict_proba(X_test_scaled)[:, 1]
test_probs_rf  = models['Random Forest'].predict_proba(X_test_scaled)[:, 1]
test_probs_xgb = models['XGBoost'].predict_proba(X_test_scaled)[:, 1]

# --- 2. 按照同样的权重进行融合 ---
# 权重必须和验证集上一样：XGBoost(0.6) + RF(0.3) + LR(0.1)
ensemble_test_preds = (0.1 * test_probs_lr + 
                       0.6 * test_probs_rf + 
                       0.3 * test_probs_xgb)

# --- 3. 生成最终提交文件 ---
submission_ensemble = pd.DataFrame({
    'id': test_df['id'],
    'Exited': ensemble_test_preds
})

# 保存为新文件，方便和单模型对比
submission_ensemble.to_csv('submission_ensemble.csv', index=False)

print("✅ 融合模型预测完成！")
print("文件已保存为: submission_ensemble.csv")
print("前 5 行预览：")
display(submission_ensemble.head())

✅ 融合模型预测完成！
文件已保存为: submission_ensemble.csv
前 5 行预览：


,id,Exited
0,15000,0.044435
1,15001,0.245009
2,15002,0.010025
3,15003,0.816130
4,15004,0.008461
